ENTRAINEMENT DU MODELE DE FORET ALEATOIRE (RANDOM FOREST)

Importation des librairies

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as nu
from sklearn.model_selection import train_test_split # Pour le split train/test
from sklearn.ensemble import RandomForestClassifier # Modèle sélectionné (catégorisation/regression)
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report # Evaluation des résultats

train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

Exploration des données

Fusion des jeux de données

In [2]:
test['Survived'] = None
full = pd.concat([train, test], sort=False)


Nettoyage des données

In [3]:
# # Extraction titre
# train['Title'] = train['Name'].str.extract(r',\s*([^\.]*)\s*\.', expand=False)
# train['Title'] = train['Title'].str.strip()
# # Verification des entrées concernées
# train[train['Age'].isnull()].groupby(['Title', 'Pclass', 'Sex', 'Parch']).size()
# # Conversion données NULL 
# age_medians = train.groupby(['Title', 'Pclass'])['Age'].median()
# # Fonction pour imputer/remplacer les âges manquants en fonction des catégories concernées
# def impute_age(row):
#     if pd.isnull(row['Age']):
#         return age_medians.get((row['Title'], row['Pclass']), train['Age'].median())
#     return row['Age']
# # Application des changements à la table
# train['Age'] = train.apply(impute_age, axis=1)
# # Conversion du type de données de la colonne 'Sex' en valeurs numériques
# from sklearn.preprocessing import LabelEncoder
# le = LabelEncoder()
# train['Sex'] = le.fit_transform(train['Sex'])

# # Conversion du type de données de la colonne 'Title' & création d'une table de correspondance pour comprendre les modifications
# train['Title'] = le.fit_transform(train['Title'])
# title_mapping = pd.DataFrame({
#     'Title': le.classes_,
#     'Encoded': le.transform(le.classes_)
# })
# title_mapping.sort_values('Encoded')

In [4]:
# Extraction titre
full['Title'] = full['Name'].str.extract(r',\s*([^\.]*)\s*\.', expand=False)
full['Title'] = full['Title'].str.strip()
# Verification des entrées concernées
full[full['Age'].isnull()].groupby(['Title', 'Pclass', 'Sex', 'Parch']).size()
# Conversion données NULL 
age_medians = full.groupby(['Title', 'Pclass'])['Age'].median()
# Fonction pour imputer/remplacer les âges manquants en fonction des catégories concernées
def impute_age(row):
    if pd.isnull(row['Age']):
        return age_medians.get((row['Title'], row['Pclass']), full['Age'].median())
    return row['Age']
# Application des changements à la table
full['Age'] = full.apply(impute_age, axis=1)
# Conversion du type de données de la colonne 'Sex' en valeurs numériques
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
full['Sex'] = le.fit_transform(full['Sex'])

# Conversion du type de données de la colonne 'Title' & création d'une table de correspondance pour comprendre les modifications
full['Title'] = le.fit_transform(full['Title'])
title_mapping = pd.DataFrame({
    'Title': le.classes_,
    'Encoded': le.transform(le.classes_)
})
title_mapping.sort_values('Encoded')

# Conversion du type de données de la colonne 'Embarked'
full['Embarked'] = LabelEncoder().fit_transform(full['Embarked'])

# Traitement des colonnes 'Embarked' et 'Fare'
full['Fare'] = full['Fare'].fillna(full['Fare'].median())
full['Embarked'] = full['Embarked'].fillna(full['Embarked'].mode()[0])




MODELE 2 - Random Forest 

In [5]:
train_cleaned = full[full['Survived'].notnull()]
test_cleaned = full[full['Survived'].isnull()]

Sélection des features (X) et de la target (y)

In [6]:
features = ['Age', 'Sex', 'Pclass', 'Title', 'Parch', 'Fare', 'SibSp']
X = train_cleaned[features]

y = train_cleaned['Survived'].astype(int)

Séparation des données d'entraînement et de test

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

Entraînement du modèle Random Forest

In [8]:
model = RandomForestClassifier(
    n_estimators=100,
    max_depth=None,
    random_state=1
)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

Prédiction des résultats sur test_cleaned

In [10]:
X_test_final = test_cleaned[features]
y_pred_final = model.predict(X_test_final)

Evaluation

In [11]:
print('Accuracy : ', accuracy_score(y_test, y_pred_final)) # Précision
print('Matrice de confusion : ',confusion_matrix(y_test, y_pred_final)) # Croisement entre les vraies données et celles prédites
print('Rapport de classification : ', classification_report(y_test, y_pred_final))

ValueError: Found input variables with inconsistent numbers of samples: [179, 418]

Amélioration du modèle </br>
=> Recherche de nouveaux hyperparamètres pour le modèle Random Forest </br>
=> Utilisation de la technique de cross-validation pour trouver des paramètres adéquates

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 200, 300], #nb d'arbres dans la forêt
    'max_depth': [None, 5, 10], # profondeur max des arbres
    'min_samples_split': [2, 4, 10] # nb mini. d'échantillons requis pour diviser un noeud
}

grid = GridSearchCV(RandomForestClassifier(),
                    param_grid, #appel du dictionnaire avec les paramètres à tester
                    cv=5, # validation croisée à 5 plis (division du dataset en 5 parties, 4 d'entrainement, 1 de test)
                    scoring='accuracy')

grid.fit(X_train, y_train) 

print('Meilleurs paramètres : ', grid.best_params_)
print('Score CV : ', grid.best_score_)


Meilleurs paramètres :  {'max_depth': 5, 'min_samples_split': 2, 'n_estimators': 100}
Score CV :  0.8173840244262779


Résumé </br>
Nombre d'arbres : 100 </br>
La profondeur max. des arbres (max_depth) : 5 </br>
Nombre d'échantillons requis pour la division des noeuds : 2 </br>


Test des nouveaux hyper paramètres

In [ ]:
model_opti = RandomForestClassifier(
    n_estimators= 100,
    max_depth= 5,
    min_samples_split=2,
    random_state=1,
)

model_opti.fit(X_train, y_train)

y_pred_opti = model_opti.predict(X_test)

print('Accuracy : ', accuracy_score(y_test, y_pred_opti)) # Précision
print('Matrice de confusion : ',confusion_matrix(y_test, y_pred_opti)) # Croisement entre les vraies données et celles prédites
print('Rapport de classification : ', classification_report(y_test, y_pred_opti))

Accuracy :  0.7597765363128491
Matrice de confusion :  [[93 13]
 [30 43]]
Rapport de classification :                precision    recall  f1-score   support

           0       0.76      0.88      0.81       106
           1       0.77      0.59      0.67        73

    accuracy                           0.76       179
   macro avg       0.76      0.73      0.74       179
weighted avg       0.76      0.76      0.75       179



Ca a diminué le score... </br>
Nouvel essai à partir des suggestions de ChatGPT

In [ ]:
# Utilisateur du meilleur modèle estimé
best_model = grid.best_estimator_
# Prédiction
y_pred_best = best_model.predict(X_test)
# Evaluation
print("Accuracy : ", accuracy_score(y_test, y_pred_best))
print('Matrice de confusion : ', confusion_matrix(y_test, y_pred_best))
print('Rapport de classification : ', classification_report(y_test, y_pred_best))

Accuracy :  0.7597765363128491
Matrice de confusion :  [[93 13]
 [30 43]]
Rapport de classification :                precision    recall  f1-score   support

           0       0.76      0.88      0.81       106
           1       0.77      0.59      0.67        73

    accuracy                           0.76       179
   macro avg       0.76      0.73      0.74       179
weighted avg       0.76      0.76      0.75       179

